In [4]:
from sentence_transformers import SentenceTransformer

In [6]:
embedding_model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(embedding_model_name)

In [7]:
user_question = "I just discovered the course. Can I still join it?"
embeddings = embedding_model.encode(user_question)

In [11]:
embeddings[0]

0.078222655